In [10]:
%pip install numpy
%pip install pandas plotly


import pandas as pd
from datetime import datetime

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pandas as pd

# Load the CSV data assuming '41.csv' is your filename
data = pd.read_csv('temp.csv')

# Step 1: Drop the first row (repeatedly dropping first index), maintaining your instructions
data = data.drop(index=0).reset_index(drop=True)
data = data.drop(index=0).reset_index(drop=True)

# Step 2: Create a list to hold all the rows that will later form the new DataFrame
rows = []

# Step 3: Iterate over each participant's data to create the new rows
for index, row in data.iterrows():
    pid = row['PROLIFIC_PID']
    response_id = row['ResponseId']
    group = row.get('Group')  # Assuming 'Group' is a column in your data for logic

    # Create a row for 'analogy' with initialized empty 'sketchscore'
    analogy_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'analogy',
        'sketchscore': None
    }

    # Create a row for 'baseline' with initialized empty 'sketchscore'
    baseline_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'baseline',
        'sketchscore': None
    }

    # Assign 'Order' based on 'Group'
    if group == '1':
        analogy_row['Order'] = 'first'
        baseline_row['Order'] = 'second'
    elif group == '2':
        analogy_row['Order'] = 'second'
        baseline_row['Order'] = 'first'

    # Add the created rows to the list
    rows.append(analogy_row)
    rows.append(baseline_row)

# Step 4: Create a new DataFrame from the list of rows
new_data = pd.DataFrame(rows, columns=['PROLIFIC_PID', 'ResponseId', 'Order', 'Technique', 'Time', 'PerformanceScore', 'DescriptionScore', 'ChartDifficulties', 'VARK', 'sketchscore'])

# Display the new DataFrame
print(new_data.head())

               PROLIFIC_PID         ResponseId   Order Technique  Time  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8  second   analogy   NaN   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   first  baseline   NaN   
2  672a528ae534c28289a223d6  R_2DJ8cYPPRIIdifS   first   analogy   NaN   
3  672a528ae534c28289a223d6  R_2DJ8cYPPRIIdifS  second  baseline   NaN   
4  647df23a62de26114fdcb29c  R_2TpcU74aCOHhhfD   first   analogy   NaN   

   PerformanceScore  DescriptionScore  ChartDifficulties  VARK sketchscore  
0               NaN               NaN                NaN   NaN        None  
1               NaN               NaN                NaN   NaN        None  
2               NaN               NaN                NaN   NaN        None  
3               NaN               NaN                NaN   NaN        None  
4               NaN               NaN                NaN   NaN        None  


In [12]:
rows = []

# Step 3: Iterate over each participant's data to create the new rows
for index, row in data.iterrows():
    pid = row['PROLIFIC_PID']
    response_id = row['ResponseId']
    group = row.get('Group')  # Assuming 'Group' is a column in your data for logic

    # Calculate dominant VARK modality
    vark_types = ['V', 'A', 'R', 'K']
    vark_counts = {v: 0 for v in vark_types}
    for j in range(1, 17):
        vark_col = f'VARK {j}'
        if vark_col in row and pd.notna(row[vark_col]):
            vark_values = str(row[vark_col]).split(',')
            for v in vark_values:
                v = v.strip()
                if v in vark_types:
                    vark_counts[v] += 1
    max_count = max(vark_counts.values())
    dominant_varks = [v for v, count in vark_counts.items() if count == max_count]
    dominant_vark = ''
    for v in vark_types:
        if v in dominant_varks:
            dominant_vark = v
            break

    # Create a row for 'analogy'
    analogy_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'analogy',
        'sketchscore': None,
        'VARK': dominant_vark
    }

    # Create a row for 'baseline'
    baseline_row = {
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'Technique': 'baseline',
        'sketchscore': None,
        'VARK': dominant_vark
    }

    # Assign 'Order' based on 'Group'
    if group == '1':
        analogy_row['Order'] = 'first'
        baseline_row['Order'] = 'second'
    elif group == '2':
        analogy_row['Order'] = 'second'
        baseline_row['Order'] = 'first'

    # Add the created rows to the list
    rows.append(analogy_row)
    rows.append(baseline_row)

# Step 4: Create a new DataFrame from the list of rows
new_data = pd.DataFrame(rows, columns=['PROLIFIC_PID', 'ResponseId', 'Order', 'Technique', 'Time', 'PerformanceScore', 'DescriptionScore', 'ChartDifficulties', 'VARK', 'sketchscore'])

# Display the new DataFrame
print(new_data.head())


               PROLIFIC_PID         ResponseId   Order Technique  Time  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8  second   analogy   NaN   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   first  baseline   NaN   
2  672a528ae534c28289a223d6  R_2DJ8cYPPRIIdifS   first   analogy   NaN   
3  672a528ae534c28289a223d6  R_2DJ8cYPPRIIdifS  second  baseline   NaN   
4  647df23a62de26114fdcb29c  R_2TpcU74aCOHhhfD   first   analogy   NaN   

   PerformanceScore  DescriptionScore  ChartDifficulties VARK sketchscore  
0               NaN               NaN                NaN    A        None  
1               NaN               NaN                NaN    A        None  
2               NaN               NaN                NaN    K        None  
3               NaN               NaN                NaN    K        None  
4               NaN               NaN                NaN    V        None  


In [29]:
key_columns = ['PROLIFIC_PID', 'ResponseId', 'ChartType']
timer_columns_suffix = ['First Click', 'Last Click']


rows = []

# Iterate over each participant's original data row
for index, row in data.iterrows():
    # Extract base information for each entry
    base_info = {k: row[k] for k in key_columns}

    # Collect all timer data by iterating column names
    set_counter = 0
    for timer_index in range(1, 9):  # Timer1 to Timer8
        for click_type in timer_columns_suffix:
            # Pattern to search for multiple occurrences
            timer_pattern = f'Timer{timer_index}_{click_type}'
            occurrences = (col for col in data.columns if timer_pattern in col)

            # For each occurrence in columns, fetch the value
            for timer_col in occurrences:
                if pd.notna(row[timer_col]):
                    new_row = base_info.copy()
                    new_row['TimerType'] = timer_pattern
                    new_row['Set'] = set_counter % 2 + 1
                    new_row['Value'] = row[timer_col]
                    rows.append(new_row)
                    set_counter += 1

    # Ensure only two sets are recorded per aspect
    sets_to_ensure = 2 * len(timer_columns_suffix)
    if set_counter / sets_to_ensure > 2:
        print(f"Warning: More than two set occurrences found for participant: {base_info}")

# Create a DataFrame from the list of rows
timed_click_data = pd.DataFrame(rows)

# Display the first few rows of the DataFrame
print(timed_click_data.head())
print("Total rows:", len(timed_click_data))

               PROLIFIC_PID         ResponseId ChartType           TimerType  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap  Timer1_First Click   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap  Timer1_First Click   
2  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap   Timer1_Last Click   
3  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap   Timer1_Last Click   
4  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap  Timer2_First Click   

   Set   Value  
0    1   5.175  
1    2   5.788  
2    1  34.633  
3    2  45.229  
4    1  13.761  
Total rows: 2192


In [30]:
print(timed_click_data.head())

               PROLIFIC_PID         ResponseId ChartType           TimerType  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap  Timer1_First Click   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap  Timer1_First Click   
2  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap   Timer1_Last Click   
3  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap   Timer1_Last Click   
4  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   Heatmap  Timer2_First Click   

   Set   Value  
0    1   5.175  
1    2   5.788  
2    1  34.633  
3    2  45.229  
4    1  13.761  


In [31]:
# Define key columns and timer column patterns
key_columns = ['PROLIFIC_PID', 'ResponseId', 'ChartType']
timer_columns_suffix = ['First Click', 'Last Click']

# List to store transformed row data
rows = []

# Iterate over each participant's data to extract timer values
for index, row in data.iterrows():
    # Extract base information for each entry
    base_info = {k: row[k] for k in key_columns}

    # Counter to differentiate between sets
    set_counter = 0
    
    for timer_index in range(1, 9):  # Timer1 to Timer8
        for click_type in timer_columns_suffix:
            timer_pattern = f'Timer{timer_index}_{click_type}'
            occurrences = (col for col in data.columns if timer_pattern in col)

            # Pull out each occurrence within timer column
            for timer_col in occurrences:
                if pd.notna(row[timer_col]):
                    new_row = base_info.copy()
                    new_row['TimerType'] = timer_pattern
                    visualization_technique = "analogy" if set_counter % 2 == 0 else "baseline"
                    new_row['VisualizationTechnique'] = visualization_technique
                    new_row['Value'] = row[timer_col]
                    rows.append(new_row)
                    set_counter += 1

# Create a DataFrame from the list of rows
timed_click_data = pd.DataFrame(rows)

# Sort the DataFrame by specified columns
timed_click_data.sort_values(
    by=['PROLIFIC_PID', 'ResponseId', 'ChartType', 'VisualizationTechnique', 'TimerType'],
    inplace=True
)

# Display the sorted DataFrame
print(timed_click_data.head())
print("Total rows:", len(timed_click_data))

                  PROLIFIC_PID         ResponseId       ChartType  \
1842  565c6d8cc121fe000ffc3ef5  R_1j7mZAyVE6uolgn  ButterflyChart   
1844  565c6d8cc121fe000ffc3ef5  R_1j7mZAyVE6uolgn  ButterflyChart   
1846  565c6d8cc121fe000ffc3ef5  R_1j7mZAyVE6uolgn  ButterflyChart   
1848  565c6d8cc121fe000ffc3ef5  R_1j7mZAyVE6uolgn  ButterflyChart   
1850  565c6d8cc121fe000ffc3ef5  R_1j7mZAyVE6uolgn  ButterflyChart   

               TimerType VisualizationTechnique   Value  
1842  Timer1_First Click                analogy   2.452  
1844   Timer1_Last Click                analogy    80.9  
1846  Timer2_First Click                analogy   9.951  
1848   Timer2_Last Click                analogy  51.739  
1850  Timer3_First Click                analogy  15.624  
Total rows: 2192


In [36]:
# Convert the 'Technique' column to match 'VisualizationTechnique'
new_data['VisualizationTechnique'] = new_data['Technique']

# Initialize list for storing calculated times
timing_summations = []

# Calculate time sums from 'timed_click_data'
for (pid, response_id, visualization_technique), group in timed_click_data.groupby(['PROLIFIC_PID', 'ResponseId', 'VisualizationTechnique']):
    total_time = 0
    for timer_index in range(1, 9):
        first_click_col = f'Timer{timer_index}_First Click'
        last_click_col = f'Timer{timer_index}_Last Click'

        # Extract the values for first and last clicks
        first_clicks = group[group['TimerType'] == first_click_col]['Value']
        last_clicks = group[group['TimerType'] == last_click_col]['Value']

        # Calculate time differences
        if not first_clicks.empty and not last_clicks.empty:
            total_time += sum(float(lc) + float(fc) for lc, fc in zip(last_clicks, first_clicks))

    # Append the result along with identifiers
    timing_summations.append({
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'VisualizationTechnique': visualization_technique,  # Match renamed field
        'Time': total_time
    })

# Convert timing_summations to a DataFrame
time_df = pd.DataFrame(timing_summations)

# Merge 'time_df' with 'new_data' based on matchable columns
final_data = pd.merge(new_data, time_df, on=['PROLIFIC_PID', 'ResponseId', 'VisualizationTechnique'], how='left')

# Drop the redundant 'VisualizationTechnique' column if needed
final_data.drop(columns='VisualizationTechnique', inplace=True)

# Display the final merged DataFrame
print(final_data.head())

               PROLIFIC_PID         ResponseId   Order Technique  Time_x  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8  second   analogy     NaN   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   first  baseline     NaN   
2  672a528ae534c28289a223d6  R_2DJ8cYPPRIIdifS   first   analogy     NaN   
3  672a528ae534c28289a223d6  R_2DJ8cYPPRIIdifS  second  baseline     NaN   
4  647df23a62de26114fdcb29c  R_2TpcU74aCOHhhfD   first   analogy     NaN   

   PerformanceScore  DescriptionScore  ChartDifficulties VARK sketchscore  \
0               NaN               NaN                NaN    A        None   
1               NaN               NaN                NaN    A        None   
2               NaN               NaN                NaN    K        None   
3               NaN               NaN                NaN    K        None   
4               NaN               NaN                NaN    V        None   

    Time_y  
0  486.166  
1  642.688  
2  868.586  
3  344.838  
4  967.017  


In [38]:
new_data['VisualizationTechnique'] = new_data['Technique']

# Initialize a list to store timing summations
timing_summations = []

# Calculate time sums and ensure 'ChartType' is included
for (pid, response_id, chart_type, technique), group in timed_click_data.groupby(['PROLIFIC_PID', 'ResponseId', 'ChartType', 'VisualizationTechnique']):
    total_time = 0
    for timer_index in range(1, 9):
        first_click_col = f'Timer{timer_index}_First Click'
        last_click_col = f'Timer{timer_index}_Last Click'

        # Extract values and calculate sums
        first_clicks = group[group['TimerType'] == first_click_col]['Value']
        last_clicks = group[group['TimerType'] == last_click_col]['Value']
        if not first_clicks.empty and not last_clicks.empty:
            total_time += sum(float(lc) - float(fc) for lc, fc in zip(last_clicks, first_clicks))

    # Append result with 'ChartType'
    timing_summations.append({
        'PROLIFIC_PID': pid,
        'ResponseId': response_id,
        'ChartType': chart_type,
        'VisualizationTechnique': technique,
        'TotalTime': total_time
    })

# Create a DataFrame from timing_summations
time_df = pd.DataFrame(timing_summations)

# Merge 'new_data' with 'time_df' on the necessary columns
final_data = pd.merge(new_data, time_df, on=['PROLIFIC_PID', 'ResponseId', 'VisualizationTechnique'], how='left')

# Display the final DataFrame with 'ChartType' included
print(final_data.head())

               PROLIFIC_PID         ResponseId   Order Technique  Time  \
0  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8  second   analogy   NaN   
1  60f47600a931759ef3d1a505  R_80P2HAsmCHZkbq8   first  baseline   NaN   
2  672a528ae534c28289a223d6  R_2DJ8cYPPRIIdifS   first   analogy   NaN   
3  672a528ae534c28289a223d6  R_2DJ8cYPPRIIdifS  second  baseline   NaN   
4  647df23a62de26114fdcb29c  R_2TpcU74aCOHhhfD   first   analogy   NaN   

   PerformanceScore  DescriptionScore  ChartDifficulties VARK sketchscore  \
0               NaN               NaN                NaN    A        None   
1               NaN               NaN                NaN    A        None   
2               NaN               NaN                NaN    K        None   
3               NaN               NaN                NaN    K        None   
4               NaN               NaN                NaN    V        None   

  VisualizationTechnique  ChartType  TotalTime  
0                analogy    Heatmap    346.

In [39]:
final_data.drop(columns=['Time'], inplace=True)

# Display the final DataFrame to confirm changes
final_data.to_csv('dataAnalysis.csv', index=False)